In [ ]:
#https://www.tarladalal.com/RecipeAtoZ.aspx
from bs4 import BeautifulSoup
import requests
import csv
import re
import pandas as pd
Title_list=[]
Tags_list=[]
PreparationTime_list=[]
CookingTime_list=[]
TotalTime_list=[]
Makes_list=[]
Ingredients_list=[]
Method_list=[]
author=[]
desc=[]
link=[]
href_link=[]
href_link1=[]
#https://www.tarladalal.com/RecipeAtoZ.aspx
c=0
a_z=['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
c=0
d=0
sep=[]
for az in a_z:
    i=1
    c=0
    page = requests.get('https://www.tarladalal.com/RecipeAtoZ.aspx?beginswith=%s&pageindex=%d'%(az,i))
    soup=BeautifulSoup(page.text,'html')
    tag=soup('a',{'class':"respglink"})
    for t in tag:
        s=str(t)
    s1=re.search('([0-9]+)',s)
    k=int(s1.group(0))
    while i<(k+1):
        page = requests.get('https://www.tarladalal.com/RecipeAtoZ.aspx?beginswith=%s&pageindex=%d'%(az,i))
        soup=BeautifulSoup(page.text,'html')
        #print(soup.prettify)
        table=soup.find_all('div',{'class':"rcc_rcpcore"})
        for t in table:
            Title_list.append(t.find('a').text)
            href_link.append(t.find('a',href=True).get('href'))
            href_link1.append('https://www.tarladalal.com/'+t.find('a',href=True).get('href'))
            c=c+1    
        d+=1
        i+=1
    print("completed for %s with %d total count"%(az,c))
print("work completed with %d no of pages extracting the link name and link parts"%c)
spd=pd.DataFrame({'Title':Title_list,'link':href_link1,'link_1':href_link})
spd.to_csv("RamBharoseKaam_2.csv")

In [ ]:
l=0
for hr in href_link:
    l=l+1
    page = requests.get('https://www.tarladalal.com/'+hr)
    soup=BeautifulSoup(page.text,'html')
    #getting the link
    link.append('https://www.tarladalal.com/'+hr)
    tag=soup.find('div',{'id':"rcpmain"})#khelne wala tag
    #description
    desc.append(tag.find('span',{'itemprop':"description"}).text)
    #author
    author.append(tag.find('span',{'itemprop':"author"}).text)
# <span id="ctl00_cntrightpanel_lblContribby" itemprop="author" style="font-size:12pt;">Tarla Dalal</span>
    #tags
    a3=""
    a=tag.find_all('a',{'itemprop':"recipeCategory"})
    for a1 in a:
        a3=a3+(a1.text)
    Tags_list.append(a3)
    try:
        PreparationTime_list.append(tag.find('time',{'itemprop':"prepTime"}).text)
    except:
        PreparationTime_list.append("-")
    try:
        CookingTime_list.append(tag.find('time',{'itemprop':"cookTime"}).text)
    except:
        CookingTime_list.append("-")
    TotalTime_list.append(tag.find('time',{'itemprop':"totalTime"}).text)
    Ma=tag.find('span',{'itemprop':"recipeYield"}).text
    Makes_list.append(Ma[5:])
    Ingredients_list.append(tag.find('div',{'id':"rcpinglist"}).text.strip())
    Ma=tag.find('ol',{'itemprop':"recipeInstructions"}).text
    if re.search('^Method',Ma)!=None:
        Method_list.append(Ma[6:])
    else:
        Method_list.append(Ma)
    print("completed for %d page"%l)
print("completed Scraping Now creating Data frame followed by Csv File")

data=pd.DataFrame({'title':Title_list,'author':author,'description':desc,'Tags':Tags_list,'Prepration_time':PreparationTime_list,'CookingTime':CookingTime_list,'TotalTime':TotalTime_list,'Make':Makes_list,'Ingredients':Ingredients_list,'Method':Method_list,'link':link})
data.to_csv('sabkakhana_fullLAst.csv',index=False)
print("Alloted Work Completed")